# Spotify Data Exploration and Modeling
## Jacob Torres

In [1]:
# Imports
import os
import re
import sqlite3
import numpy as np
import pandas as pd
import tensorflow as tf
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyOAuth
from spotipy import Spotify

## Exploratory Data Analysis
---

In [2]:
# Load Spotify data from local db
DATA_TABLE_QUERY = 'select * from data;'
conn = sqlite3.connect('../app/spotify_db.sqlite3')

# Dataframe of unliked songs
unliked_df = pd.read_sql(DATA_TABLE_QUERY, conn)
unliked_df = unliked_df.sample(5000)

print(f"Unliked songs: {unliked_df.shape}")
unliked_df.head()

Unliked songs: (5000, 14)


,name,id,acousticness,danceability,duration_ms,energy,tempo,instrumentalness,key,liveness,loudness,mode,valence,speechiness
111381,I Like Them All,4Fzc2djdWBQYbmenQntley,0.896,0.593,165333,0.196,153.311,0.000000,9,0.2100,-15.483,0,0.885,0.0658
118690,Contra Viento Y Marea,3VNYBo5b2n5rbCllOQ9ft1,0.165,0.727,175840,0.489,120.544,0.000000,1,0.1010,-9.752,0,0.823,0.0291
100960,9th & Hennepin,2fScFKA4WWLTFWVoJ1UQW7,0.831,0.584,116867,0.155,137.858,0.000062,9,0.1510,-20.666,0,0.470,0.1190
129604,Ask Me,36B3wTZu6VFok9MHYrv9yd,0.857,0.365,126973,0.274,79.081,0.962000,4,0.1690,-14.778,0,0.355,0.0308
123080,Get It On Tonite,0AcLrSfAEBQcUnHOTm5pXg,0.259,0.807,277413,0.498,99.001,0.000030,10,0.0618,-9.910,0,0.857,0.0756


In [3]:
# Create "unliked" vector for later use in the dataframe
unliked_vect = list(
    np.zeros(unliked_df.shape[0], np.int64)
)

### Authorize Spotify Access for Further Data Collection

In [4]:
# Authorize access to Spotify API
load_dotenv()
cid = os.getenv('CLIENT_ID')
cs = os.getenv('CLIENT_SECRET')
rduri = os.getenv('REDIRECT_URI')
scope = [
    'playlist-read-private',
    'playlist-modify-public',
    'user-library-read',
    'user-read-recently-played'
]

assert cid is not None and cs is not None and rduri is not None, 'One or more environment variables are missing.'

In [5]:
auth_manager = SpotifyOAuth(
    client_id=cid,
    client_secret=cs,
    redirect_uri=rduri,
    scope=scope
)
sp = Spotify(auth_manager=auth_manager)

sp

### Data Collection

In [6]:
# Functions for data collection via the API
def get_tracks(num=50, index=0):
    """
    Get tracks and track details from current user library starting at given index position.

    returns ids, names, tracks, audio_features -- track details
    """
    tracks = sp.current_user_saved_tracks(limit=num, offset=index)['items']
    names = [track['track']['name'] for track in tracks]
    ids = [track['track']['id'] for track in tracks]
    audio_features = []

    try:
        for id in ids:
            audio_features.append(
                sp.audio_features(id)
            )

        return ids, names, tracks, audio_features

    except:
        return ids, names, tracks, None


def get_new_likes(num=50, index=0):
    """
    Collects features for a number of tracks, starting at given index.

    returns new_likes -- dataframe of num liked tracks and audio features
    """
    ids, names, tracks, audio_features = get_tracks(num=num, index=index)

    # Create dataframe of new likes and audio features
    columns = [
        'name',
        'id',
        'duration_ms',
        'danceability',
        'energy',
        'key',
        'loudness',
        'mode',
        'speechiness',
        'acousticness',
        'instrumentalness',
        'liveness',
        'valence',
        'tempo'
    ]

    new_likes = pd.DataFrame(columns=columns)

    # Create list of track dicts with audio features
    if audio_features is None:
        print('Something went wrong.')

    elif len(audio_features) > 0:
        track_data = []
        for i, features in enumerate(audio_features):
            track = {
                'name': names[i],
                'id': features[0]['id'],
                'duration_ms': features[0]['duration_ms'],
                'danceability': features[0]['danceability'],
                'energy': features[0]['energy'],
                'key': features[0]['key'],
                'loudness': features[0]['loudness'],
                'mode': features[0]['mode'],
                'speechiness': features[0]['speechiness'],
                'acousticness': features[0]['acousticness'],
                'instrumentalness': features[0]['instrumentalness'],
                'liveness': features[0]['liveness'],
                'valence': features[0]['valence'],
                'tempo': features[0]['tempo']
            }

            track_data.append(track)

        new_likes = new_likes.append(
            track_data,
            ignore_index=True,
            verify_integrity=True
        )

    return new_likes

In [7]:
ids, names, tracks, audio_features = get_tracks()

assert (
    len(ids) == 50 and
    len(names) == 50 and
    len(tracks) == 50 and
    len(audio_features) != 0
), 'Something went wrong.'

In [8]:
# Get audio features of the tracks
audio_features[0]
#[print(i, features[0]['id']) for i, features in enumerate(audio_features)]

[{'danceability': 0.734,
  'energy': 0.712,
  'key': 6,
  'loudness': -6.217,
  'mode': 0,
  'speechiness': 0.201,
  'acousticness': 0.391,
  'instrumentalness': 0,
  'liveness': 0.103,
  'valence': 0.674,
  'tempo': 140.369,
  'type': 'audio_features',
  'id': '4CPcZHzSvIBM8AKhdHUwnL',
  'uri': 'spotify:track:4CPcZHzSvIBM8AKhdHUwnL',
  'track_href': 'https://api.spotify.com/v1/tracks/4CPcZHzSvIBM8AKhdHUwnL',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4CPcZHzSvIBM8AKhdHUwnL',
  'duration_ms': 165405,
  'time_signature': 4}]

### Add New TrackFeatures to Liked Song Dataset

In [9]:
# Add liked tracks to dataframe
indices = np.random.randint(0, 3000, 200)
liked_df = get_new_likes()

for index in indices:
    new_likes = get_new_likes(index=index)
    liked_df = liked_df.append(new_likes, ignore_index=True, verify_integrity=True)

print(liked_df.shape)
liked_df.tail(50)

(2435, 14)


,name,id,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
2385,Sunshine On My Shoulders,3m8lTUL5GxGUNSaycI77ND,311893,0.322,0.1680,10,-13.893,1,0.0336,0.915000,0.000223,0.1170,0.2590,146.680
2386,Told You So,1u76oCKjI9Wkb7DA6Q5LRV,239480,0.613,0.6270,5,-6.626,1,0.0503,0.186000,0.000000,0.0541,0.5060,67.489
2387,It's Over,6KLAVFlcTrUt9IteOXzsT8,246907,0.593,0.6790,1,-4.680,0,0.0507,0.284000,0.000000,0.1520,0.2070,135.931
2388,Beautiful Soul,1HwpWwa6bnqqRhK8agG4RS,214227,0.660,0.6660,9,-4.342,1,0.0472,0.075900,0.000000,0.0268,0.9330,89.975
2389,Black Me Out,6XI33hctgDLUIRKI8wsJS4,187813,0.520,0.8850,4,-4.432,1,0.0371,0.001440,0.000007,0.1210,0.6020,104.646
2390,Two Coffins,5yDqUTEQVuhE3TT3l2ltlB,140187,0.545,0.7070,3,-5.302,1,0.0386,0.241000,0.000000,0.3680,0.6070,166.692
2391,Landslide,5ihS6UUlyQAfmp48eSkxuQ,199493,0.414,0.1610,7,-22.320,0,0.0318,0.883000,0.000223,0.1170,0.4230,159.375
2392,The Chain - 2004 Remaster,5e9TFTbltYBg2xThimr0rU,270213,0.545,0.6700,9,-8.810,1,0.0496,0.009000,0.000822,0.0451,0.4810,151.553
2393,Two Ghosts,4B1rpPmQXwj78wk6aIGwwU,229813,0.386,0.4070,6,-7.095,0,0.0268,0.234000,0.000002,0.0999,0.4560,69.387
2394,"Sunflower, Vol. 6",6iYMfxznTBlcVOgRHab2W0,221827,0.566,0.8460,7,-6.904,0,0.0350,0.000564,0.000795,0.1210,0.8600,160.020


In [13]:
# Create "liked" vector
liked_vect = list(
    np.ones(liked_df.shape[0], np.int64)
)

In [14]:
# Combine liked and unliked tracks, and add "liked" column
df = unliked_df.append(liked_df, ignore_index=True, verify_integrity=True)
df['liked'] = unliked_vect + liked_vect

print(df.shape)
df['liked'].value_counts()

(7435, 15)


0    5000
1    2435
Name: liked, dtype: int64

In [15]:
df.sample(50)

,name,id,acousticness,danceability,duration_ms,energy,tempo,instrumentalness,key,liveness,loudness,mode,valence,speechiness,liked
3111,Separarla También,2uUimJq18JcrZjgkuvLuMa,0.740000,0.483,155634,0.7210,85.362,0.790000,7,0.1000,-5.899,1,0.9450,0.0563,0
4949,El Beso de la Muerte - Remasterizado,5wg1nnx7c2kri8kx5xlCFH,0.996000,0.493,158587,0.0997,127.023,0.481000,5,0.1920,-23.446,1,0.5870,0.0607,0
142,Chaman Mein Rah Ke Virana,4oxWVUHYUy6WAiTV0m3wm2,0.886000,0.622,193507,0.4890,125.977,0.000021,2,0.0856,-5.190,1,0.5110,0.0702,0
1998,Backroad Song,6naYpFIKihKbGg6lGxpveI,0.035200,0.488,236787,0.8330,174.006,0.017200,11,0.3400,-6.571,1,0.7510,0.0437,0
1532,Cruel,3j5DVpcCELigVZrmwGOw3X,0.010400,0.570,214987,0.7040,115.041,0.455000,1,0.0905,-8.320,1,0.4370,0.0314,0
3293,My Funny Valentine - Live at Philharmonic Hall...,176PPnVoT7ZnAl7urhkmoa,0.838000,0.492,900933,0.1490,128.094,0.578000,8,0.9700,-15.895,1,0.2540,0.0444,0
1836,Band Of Gold - Re-Record,0YgoNapnyIXuruBrzvoCSu,0.451000,0.631,182533,0.7460,108.875,0.000000,7,0.4240,-5.439,1,0.9300,0.0277,0
1560,"Sonata No. 5, Op. 24 ""Spring"" in F: Allegro",42ScjPBuyHEohFNxsxKmYX,0.956000,0.335,395427,0.1700,75.815,0.561000,0,0.0780,-15.961,1,0.3430,0.0454,0
6776,Like a Boy,358bOvBiZCS9fRzNYosw6c,0.267000,0.701,237053,0.7240,132.035,0.000000,0,0.0867,-5.751,0,0.4250,0.1450,1
4272,Not Strong Enough,1mCNkrKh9NEgxJWUqHnEgo,0.001830,0.354,216520,0.8290,163.958,0.000000,9,0.1190,-3.834,1,0.1950,0.0470,0


In [16]:
df.describe()

,acousticness,danceability,energy,tempo,instrumentalness,liveness,loudness,valence,speechiness,liked
count,7435.000000,7435.000000,7435.000000,7435.000000,7435.000000,7435.000000,7435.000000,7435.000000,7435.000000,7435.000000
mean,0.473059,0.566529,0.492669,116.659233,0.139648,0.193386,-10.689551,0.523740,0.095100,0.327505
std,0.358654,0.170657,0.252507,30.129818,0.284622,0.159141,5.248577,0.254532,0.146401,0.469335
min,0.000001,0.000000,0.000020,0.000000,0.000000,0.017500,-41.733000,0.000000,0.000000,0.000000
25%,0.113000,0.453000,0.285000,93.834000,0.000000,0.098600,-13.588000,0.323000,0.034700,0.000000
50%,0.452000,0.579000,0.487000,115.003000,0.000151,0.128000,-9.811000,0.523000,0.045400,0.000000
75%,0.833000,0.693000,0.696000,135.404000,0.057900,0.240500,-6.716000,0.734000,0.079600,1.000000
max,0.996000,0.978000,0.999000,217.032000,1.000000,0.998000,0.878000,1.000000,0.969000,1.000000


## Data Prep and Modeling
---